In [1]:
import sys
sys.path.insert(0, '..')

import torch
import os
import wandb
import random
import numpy as np
import torch
from torch import nn
from torch.optim import Adam
from ignite.handlers.param_scheduler import create_lr_scheduler_with_warmup
from torch.utils.data import DataLoader
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from tqdm import tqdm
from matplotlib import cm
import seaborn as sns
import matplotlib.lines as mlines
from sklearn.decomposition import PCA
from PIL import Image
import umap
import torch.nn.functional as F
from scipy.spatial.distance import cdist
import math
from scipy.stats import norm
import joblib
import matplotlib.colors as mcolors
import matplotlib.image as mpimg
import cairosvg
from io import BytesIO

from core.dataset import PSMDataset
from core.model import GalSpecNet, MetaModel, Informer, AstroM3
from core.trainer import Trainer

In [2]:
def get_model(config):
    if config['mode'] == 'photo':
        model = Informer(config)
    elif config['mode'] == 'spectra':
        model = GalSpecNet(config)
    elif config['mode'] == 'meta':
        model = MetaModel(config)
    else:
        model = AstroM3(config)

    return model

def get_embs(dataloader):
    all_p_emb, all_s_emb, all_m_emb = [], [], []
    all_labels = []
    
    for photometry, photometry_mask, spectra, metadata, labels in tqdm(dataloader):
        photometry, photometry_mask = photometry.to(device), photometry_mask.to(device)
        spectra, metadata = spectra.to(device), metadata.to(device)
    
        with torch.no_grad():
            p_emb, s_emb, m_emb = model.get_embeddings(photometry, photometry_mask, spectra, metadata)
    
            all_p_emb.append(p_emb.cpu())
            all_s_emb.append(s_emb.cpu())
            all_m_emb.append(m_emb.cpu())
            all_labels.append(labels)
    
    all_p_emb = torch.vstack(all_p_emb)
    all_s_emb = torch.vstack(all_s_emb)
    all_m_emb = torch.vstack(all_m_emb)
    all_labels = torch.hstack(all_labels)

    return all_p_emb, all_s_emb, all_m_emb, all_labels

def get_centers(p_emb, s_emb, m_emb, train_labels):
    p_centers, s_centers, m_centers, all_centers = [], [], [], []

    for i in range(10):
        ind = train_labels == i
    
        p_center = p_emb[ind].mean(axis=0)
        p_center = p_center / p_center.norm()
        p_centers.append(p_center)
    
        s_center = s_emb[ind].mean(axis=0)
        s_center = s_center / s_center.norm()
        s_centers.append(s_center)
    
        m_center = m_emb[ind].mean(axis=0)
        m_center = m_center / m_center.norm()
        m_centers.append(m_center)

        all_emb = (p_emb + s_emb + m_emb) / 3
        all_center = all_emb[ind].mean(axis=0)
        all_center = all_center / all_center.norm()
        all_centers.append(all_center)
    
    p_centers = torch.stack(p_centers)
    s_centers = torch.stack(s_centers)
    m_centers = torch.stack(m_centers)
    all_centers = torch.stack(all_centers)

    return p_centers, s_centers, m_centers, all_centers

def plot_obj_umap(dataset, idx, embeddings, targets):
    el = dataset.df.iloc[idx]
    label = target2id[el['target']]
    period = el['org_period']
    l, b = id2lb[el['id']]
    
    photometry = dataset.get_vlc(el['name'])
    photometry = np.vstack(((photometry[:, 0] % period) / period, photometry[:, 1], photometry[:, 2])).T    # fold
    spectra = dataset.readLRSFits(os.path.join(dataset.lamost_spec_dir, el['spec_filename']))

    obj_embedding = embeddings[idx]
    
    fig, axs = plt.subplots(1, 3, figsize=(24, 8))
    axs[0].plot(photometry[:, 0], photometry[:, 1], '.', label='Photometry')
    axs[1].plot(spectra[:, 0], spectra[:, 1], label='Spectra')
    
    for class_name in sorted_classes:
        class_id = target2id[class_name]
        class_mask = targets == class_id
        axs[2].scatter(embeddings[class_mask, 0], embeddings[class_mask, 1], 
                       color=palette[class_id], label=class_name, alpha=1.0, s=marker_size)

    axs[2].scatter(obj_embedding[0], obj_embedding[1], color=palette[targets[idx]], edgecolors='black', alpha=1.0, s=5 * marker_size)
    handles_colors = [mlines.Line2D([], [], color=palette[target2id[class_name]], marker='o', linestyle='None', markersize=8, 
                      label=class_name) for class_name in sorted_classes]
    axs[2].legend(handles=handles_colors, loc='upper right', bbox_to_anchor=(1.15, 1), fontsize=10, title="Classes")
    
    
    plt.suptitle(f'period = {period}    label = {id2target[label]}     idx {idx}    L = {l}    B = {b}    id {el["id"]}')
    plt.tight_layout()
    plt.show()

In [3]:
org_train = pd.read_csv('/home/mariia/AstroML/data/asassn/preprocessed_data/full_lb/spectra_and_v_train.csv')
org_val = pd.read_csv('/home/mariia/AstroML/data/asassn/preprocessed_data/full_lb/spectra_and_v_val.csv')
org_test = pd.read_csv('/home/mariia/AstroML/data/asassn/preprocessed_data/full_lb/spectra_and_v_test.csv')

org_train = org_train[['id', 'l', 'b']]
org_val = org_val[['id', 'l', 'b']]
org_test = org_test[['id', 'l', 'b']]

combined_df = pd.concat([org_train, org_val, org_test])
id2lb = combined_df.set_index('id')[['l', 'b']].T.to_dict('list')

run_id = 'MeriDK/AstroCLIPResults3/2wz4ysvn'
api = wandb.Api()
run = api.run(run_id)
config = run.config
config['use_wandb'] = False

model = get_model(config)
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

weights_path = os.path.join(config['weights_path'] + '-' + run_id.split('/')[-1], f'weights-best.pth')
model.load_state_dict(torch.load(weights_path, weights_only=False))

In [4]:
config['file'] = 'preprocessed_data/sub50_lb/spectra_and_v'

train_dataset = PSMDataset(config, split='train')
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=False)

val_dataset = PSMDataset(config, split='val')
val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

test_dataset = PSMDataset(config, split='test')
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)

train_p_emb, train_s_emb, train_m_emb, train_labels = get_embs(train_dataloader)
val_p_emb, val_s_emb, val_m_emb, val_labels = get_embs(val_dataloader)
test_p_emb, test_s_emb, test_m_emb, test_labels = get_embs(test_dataloader)

p_centers, s_centers, m_centers, all_centers = get_centers(train_p_emb, train_s_emb, train_m_emb, train_labels)

train_emb = (train_p_emb + train_s_emb + train_m_emb) / 3
val_emb = (val_p_emb + val_s_emb + val_m_emb) / 3
test_emb = (test_p_emb + test_s_emb + test_m_emb) / 3

umap = joblib.load('umap.pkl')
train_umap = umap.transform(train_emb)
val_umap = umap.transform(val_emb)
test_umap = umap.transform(test_emb)

In [5]:
config['file'] = 'preprocessed_data/full_lb/spectra_and_v'
config['classes'] = ['LSP', 'ROT:', 'DCEP', 'VAR', 'SRD', 'DCEPS', 'RRD', 'CWA', 'CWB']

new_train = PSMDataset(config, split='train')
new_train_dataloader = DataLoader(new_train, batch_size=config['batch_size'], shuffle=False)
new_train_p_emb, new_train_s_emb, new_train_m_emb, new_train_labels = get_embs(new_train_dataloader)
new_train_emb = (new_train_p_emb + new_train_s_emb + new_train_m_emb) / 3
new_train_umap = umap.transform(new_train_emb)

In [6]:
class_freq = test_dataset.df['target'].value_counts()
sorted_classes = class_freq.index

id2target = test_dataset.id2target
target2id = test_dataset.target2id

palette = sns.color_palette("Spectral", len(id2target))
palette[3], palette[-4] = palette[-4], palette[3] 
palette[1], palette[-2] = palette[-2], palette[1] 
marker_size = 12

new_id2target = new_train.id2target
new_target2id = new_train.target2id
new_palette = sns.color_palette("hls", len(new_id2target))
new_palette[3], new_palette[-1] = new_palette[-1], new_palette[3] 

In [7]:
new_palette

In [19]:
def plot_one_embs_new_classes(embeddings, targets, new_embeddings, new_targets):
    plt.figure(figsize=(10, 8))
    
    plt.scatter(embeddings[:, 0], embeddings[:, 1], c=targets, cmap=mcolors.ListedColormap(palette), alpha=0.3, s=marker_size)
    plt.scatter(new_embeddings[:, 0], new_embeddings[:, 1], c=new_targets, cmap=mcolors.ListedColormap(new_palette), 
                edgecolors='black', alpha=1.0, s=3 * marker_size)

    unique_labels = np.unique(targets)
    handles_colors = [mlines.Line2D([], [], color=palette[label], marker='o', linestyle='None', markersize=8, 
                    label=id2target[label]) for label in unique_labels]
    
    unique_new_labels = np.unique(new_targets)
    new_handles_colors = [mlines.Line2D([], [], color=new_palette[label], marker='o', linestyle='None', markersize=8, 
                                        label=new_id2target[label]) for label in unique_new_labels]

    combined_legend = handles_colors + new_handles_colors
    plt.legend(handles=combined_legend, loc='upper right', bbox_to_anchor=(1.22, 1), fontsize=14, title="Classes")
    
    plt.xlabel("UMAP Dimension 1", fontsize=14)
    plt.ylabel("UMAP Dimension 2", fontsize=14)
    plt.show()

In [20]:
plot_one_embs_new_classes(train_umap, train_labels, new_train_umap, new_train_labels)

In [8]:
def plot_one_embs_new_classes2(embeddings, targets, new_embeddings, new_targets):
    plt.figure(figsize=(10, 8))
    
    # Plot old embeddings
    plt.scatter(embeddings[:, 0], embeddings[:, 1], c=targets, cmap=mcolors.ListedColormap(palette), alpha=0.3, s=marker_size)
    
    # Plot new embeddings with a black edge for distinction
    plt.scatter(new_embeddings[:, 0], new_embeddings[:, 1], c=new_targets, cmap=mcolors.ListedColormap(new_palette), 
                edgecolors='black', alpha=1.0, s=3 * marker_size)
    
    # Generate handles for old classes' legend
    unique_labels = np.unique(targets)
    handles_colors_old = [mlines.Line2D([], [], color=palette[label], marker='o', linestyle='None', markersize=8, 
                                        label=id2target[label]) for label in unique_labels]
    
    # Generate handles for new classes' legend
    unique_new_labels = np.unique(new_targets)
    handles_colors_new = [mlines.Line2D([], [], color=new_palette[label], marker='o', linestyle='None', markersize=8, 
                                        label=new_id2target[label]) for label in unique_new_labels]
    
    # Set a fixed width for both legends
    legend_width = 0.25  # Adjust this value as needed
    
    # Create legends with consistent width settings
    legend_old = plt.legend(handles=handles_colors_old, loc='upper left', bbox_to_anchor=(1.0, 0.943, legend_width, 0.1), 
                            fontsize=14, title="Classes", title_fontsize=14, handlelength=2, borderaxespad=1.2,
                            mode='expand')
    legend_new = plt.legend(handles=handles_colors_new, loc='upper left', bbox_to_anchor=(1.0, 0.404, legend_width, 0.1), 
                            fontsize=14, title="Unseen", title_fontsize=14, handlelength=2, borderaxespad=1.2,
                            mode='expand')
    
    # Add the old classes legend back to the plot to avoid overlap
    plt.gca().add_artist(legend_old)
    
    # Labels for axes with font size 14
    plt.xlabel("UMAP Dimension 1", fontsize=14)
    plt.ylabel("UMAP Dimension 2", fontsize=14)
    plt.savefig("unseen.svg", format="svg")
    plt.show()

In [9]:
plot_one_embs_new_classes2(train_umap, train_labels, new_train_umap, new_train_labels)

In [10]:
svg_path = 'unseen.svg'
png_data = BytesIO()
cairosvg.svg2png(url=svg_path, write_to=png_data)
png_data.seek(0)

# Display the image using Matplotlib
img = mpimg.imread(png_data)
plt.imshow(img)
plt.axis('off')  # Hide axes for a cleaner look
plt.show()

In [31]:
scatter = plt.scatter(new_train_umap[:, 0], new_train_umap[:, 1], c=new_train_labels, cmap='viridis', alpha=1.0)

unique_labels = np.unique(new_train_labels)
handles = [plt.Line2D([0], [0], marker='o', color='w', label=new_train.id2target[label],
                      markersize=10, markerfacecolor=plt.cm.viridis(scatter.norm(label))) 
           for label in unique_labels]

plt.legend(handles=handles, title="Classes", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [15]:
plot_obj_umap(test_dataset, 120, test_umap, test_labels)

In [28]:
new_train.id2target

In [10]:
df = pd.read_csv('/home/mariia/AstroML/data/asassn/preprocessed_data/full_lb/spectra_and_v_val.csv')

In [11]:
df['target'].value_counts()